In [14]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *

warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

In [19]:
schema = StructType([StructField('Id', IntegerType(), True),
           StructField('SepalLengthCm', DoubleType(), True),
           StructField('SepalWidthCm', DoubleType(), True),
           StructField('PetalLengthCm', DoubleType(), True),
           StructField('PetalWidthCm', DoubleType(), True),
           StructField('Species', StringType(), True)])

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/22 13:03:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("CREATE TABLE IF NOT EXISTS iris (Id INT, SepalLengthCm DOUBLE, SepalWidthCm DOUBLE, PetalLengthCm DOUBLE, PetalWidthCm DOUBLE, Species STRING) USING hive")

21/12/22 13:04:13 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/22 13:04:13 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/12/22 13:04:16 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/12/22 13:04:16 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/12/22 13:04:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/22 13:04:16 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


DataFrame[]

In [20]:
df = spark.read.format("CSV").option("header","true").schema(schema).load('hdfs://hdfs:9000/user/Iris.csv')

In [21]:
df.show()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

In [22]:
df.dtypes

[('Id', 'int'),
 ('SepalLengthCm', 'double'),
 ('SepalWidthCm', 'double'),
 ('PetalLengthCm', 'double'),
 ('PetalWidthCm', 'double'),
 ('Species', 'string')]

In [23]:
df.write.insertInto('iris')

21/12/22 15:14:59 ERROR KeyProviderCache: Could not find uri with key [dfs.encryption.key.provider.uri] to create a keyProvider !!
21/12/22 15:14:59 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [25]:
spark.sql('select max(PetalWidthCm) from iris').show()

+-----------------+
|max(PetalWidthCm)|
+-----------------+
|              2.5|
+-----------------+



In [26]:
!jupyter nbconvert --to script 3.simple-hive.ipynb

[NbConvertApp] Converting notebook 3.simple-hive.ipynb to script
[NbConvertApp] Writing 1247 bytes to 3.simple-hive.py


In [27]:
!/opt/spark/bin/spark-submit \
  --master spark://master:7077 \
  /home/data/3.simple-hive.py

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/12/22 15:17:14 INFO SparkContext: Running Spark version 3.2.0
21/12/22 15:17:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/22 15:17:15 INFO ResourceUtils: ==============================================================
21/12/22 15:17:15 INFO ResourceUtils: No custom resources configured for spark.driver.
21/12/22 15:17:15 INFO ResourceUtils: ==============================================================
21/12/22 15:17:15 INFO SparkContext: Submitted application: Python Spark SQL Hive integration example
21/12/22 15:17:15 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21

21/12/22 15:17:33 INFO HiveMetaStore: 0: create_table: Table(tableName:iris, dbName:default, owner:root, createTime:1640186244, lastAccessTime:0, retention:0, sd:StorageDescriptor(cols:[FieldSchema(name:Id, type:int, comment:null), FieldSchema(name:SepalLengthCm, type:double, comment:null), FieldSchema(name:SepalWidthCm, type:double, comment:null), FieldSchema(name:PetalLengthCm, type:double, comment:null), FieldSchema(name:PetalWidthCm, type:double, comment:null), FieldSchema(name:Species, type:string, comment:null)], location:hdfs://hdfs:9000/user/warehouse/iris, inputFormat:org.apache.hadoop.mapred.TextInputFormat, outputFormat:org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat, compressed:false, numBuckets:-1, serdeInfo:SerDeInfo(name:null, serializationLib:org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, parameters:{serialization.format=1}), bucketCols:[], sortCols:[], parameters:{}, skewedInfo:SkewedInfo(skewedColNames:[], skewedColValues:[], skewedColValueLocationMaps

21/12/22 15:17:40 INFO DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 4.359156 s
21/12/22 15:17:40 INFO CodeGenerator: Code generated in 67.370373 ms
+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|    

21/12/22 15:17:42 INFO FileFormatWriter: Write Job 47f0c6d7-68cf-4c33-a14c-5b3276aa0a37 committed. Elapsed time: 66 ms.
21/12/22 15:17:42 INFO FileFormatWriter: Finished processing stats for write job 47f0c6d7-68cf-4c33-a14c-5b3276aa0a37.
21/12/22 15:17:42 INFO HiveMetaStore: 0: get_table : db=default tbl=iris
21/12/22 15:17:42 INFO audit: ugi=root	ip=unknown-ip-addr	cmd=get_table : db=default tbl=iris	
21/12/22 15:17:42 INFO HiveMetaStore: 0: get_table : db=default tbl=iris
21/12/22 15:17:42 INFO audit: ugi=root	ip=unknown-ip-addr	cmd=get_table : db=default tbl=iris	
21/12/22 15:17:42 ERROR KeyProviderCache: Could not find uri with key [dfs.encryption.key.provider.uri] to create a keyProvider !!
21/12/22 15:17:43 INFO HiveMetaStore: 0: alter_table: db=default tbl=iris newtbl=iris
21/12/22 15:17:43 INFO audit: ugi=root	ip=unknown-ip-addr	cmd=alter_table: db=default tbl=iris newtbl=iris	
21/12/22 15:17:43 INFO log: Updating table stats fast for iris
21/12/22 15:17:43 INFO log: Updated s

21/12/22 15:17:45 INFO TaskSetManager: Starting task 0.0 in stage 4.0 (TID 5) (172.20.0.5, executor 0, partition 0, NODE_LOCAL, 4457 bytes) taskResourceAssignments Map()
21/12/22 15:17:45 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on 172.20.0.5:41523 (size: 5.8 KiB, free: 366.3 MiB)
21/12/22 15:17:45 INFO MapOutputTrackerMasterEndpoint: Asked to send map output locations for shuffle 0 to 172.20.0.5:34254
21/12/22 15:17:46 INFO TaskSetManager: Finished task 0.0 in stage 4.0 (TID 5) in 466 ms on 172.20.0.5 (executor 0) (1/1)
21/12/22 15:17:46 INFO TaskSchedulerImpl: Removed TaskSet 4.0, whose tasks have all completed, from pool 
21/12/22 15:17:46 INFO DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 0.493 s
21/12/22 15:17:46 INFO DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
21/12/22 15:17:46 INFO TaskSchedulerImpl: Killing all running tasks in stage 4: Stage finished
21/12/22 15:17:46 